In [8]:
import pdfminer
import requests
from bs4 import BeautifulSoup
from pdfminer import pdfparser
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import string
from scipy.spatial.distance import cosine
import numpy as np
from nltk import PorterStemmer
from nltk import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
import lda

In [2]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from cStringIO import StringIO

def convert_pdf_to_txt(path):
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    fp = file(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()

    for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages,\
                                  password=password,caching=caching, check_extractable=True):
        interpreter.process_page(page)

    text = retstr.getvalue()

    fp.close()
    device.close()
    retstr.close()
    return text

In [3]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.layout import LAParams
from pdfminer.converter import  TextConverter # , XMLConverter, HTMLConverter
import urllib2
from urllib2 import Request
import datetime
import re

# Define a PDF parser function
def parsePDF(url):

    # Open the url provided as an argument to the function and read the content
    open = urllib2.urlopen(Request(url)).read()

    # Cast to StringIO object
    from StringIO import StringIO
    memory_file = StringIO(open)

    # Create a PDF parser object associated with the StringIO object
    parser = PDFParser(memory_file)

    # Create a PDF document object that stores the document structure
    document = PDFDocument(parser)

    # Define parameters to the PDF device objet 
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    laparams = LAParams()
    codec = 'utf-8'

    # Create a PDF device object
    device = TextConverter(rsrcmgr, retstr, codec = codec, laparams = laparams)

    # Create a PDF interpreter object
    interpreter = PDFPageInterpreter(rsrcmgr, device)

    # Process each page contained in the document
    for page in PDFPage.create_pages(document):
        interpreter.process_page(page)
        data =  retstr.getvalue()
    return data

In [4]:
def get_text(url):
    '''Function to pull all papers (pdf) off the arxiv'''
    base_url = url
    r = requests.get(base_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    pdfs = soup.findAll(title = 'Download PDF')
    links = [str(pdf).split()[1].strip('href="') for pdf in pdfs]
    urls = ['http://arxiv.org'+ link for link in links]
    articles = []
    for url in urls:
        articles.append(parsePDF(url))
    return articles

In [5]:
#steph = convert_pdf_to_txt('/Users/Colin/Downloads/plms.pdf')
#physics1 = convert_pdf_to_txt('/Users/Colin/Downloads/physics1.pdf')
#shake = convert_pdf_to_txt('/Users/Colin/Downloads/shakespeare-tempest.pdf')
#steph2 = convert_pdf_to_txt('/Users/Colin/Downloads/twists_final.pdf')

In [6]:
text= []
%time text = get_text('http://arxiv.org/list/math.NT/1508?show=150')


CPU times: user 8min 5s, sys: 2.21 s, total: 8min 7s
Wall time: 40min 13s


In [76]:
#Try to get rid of weird unicode characters
S = set()
S.update(letter for letter in string.lowercase)
S.update(letter for letter in string.uppercase)
S.update(digit for digit in string.digits)

In [77]:
#text.append(steph)
#text.append(shake)

#f = lambda x: 1 if x in string.printable else 0
#goodwords = []
#words = ['abcd\xce\xc2fg', 'running', 'primes', 'groups']
#for i in range(len(text)):
 #   goodwords.append([word for word in text[i].split() if all([f(char) for char in word])])


new_text=[]
for i in range(len(text)):
    new_text.append([word for word in text[i].split() if (word[0] in S) and (word[-1] in S)\
                    and (len(word)>3)])
   
for i in range(len(new_text)):
    new_text[i] = ' '.join(new_text[i])

In [10]:
#Make stop list
tfidf = TfidfVectorizer(stop_words='english')
Stop= set()
Stop.update([word for word in tfidf.get_stop_words()])
Stop.update(['theorem', 'lemma', 'proof', 'sum', 'difference', \
             'product', 'multiple', 'let', 'group', 'prime', 'log', 'limit', 'cid', 'result'\
            'main', 'conjecture', 'case', 'suppose', 'function', 'assume', 'follows', \
            'given', 'define', 'note', 'defined', 'class', 'proposition', 'function', 'set', \
             'primes', 'numbers','form', 'integers', 'curves', 'real'])
Stop = list(Stop)

In [87]:
#new_text.append(new_steph)
tfidf_math = TfidfVectorizer(max_features=100, stop_words=Stop, ngram_range=(2,2), decode_error='ignore')
M = tfidf_math.fit_transform(new_text)

In [88]:
#Get some topics

feature_names = tfidf_math.get_feature_names()
nmf = NMF(n_components=10)
nmf.fit(M)
for topic_idx, topic in enumerate(nmf.components_):
    print("Topic #%d:" % topic_idx)
    print(" ".join([feature_names[i]
                    for i in topic.argsort()[:-10 - 1:-1]]))
    print '\n'

Topic #0:
lower bound upper bound suﬃciently large probability measure section prove number theory 2n 2n non zero algebraic number absolute value


Topic #1:
elliptic curve abelian variety modular forms abelian varieties algebraic closure lecture notes polynomial degree absolute value positive integer quadratic space


Topic #2:
12 12 mean value error term arithmetic properties right hand functional equation eisenstein series 2n 2n algebraic number cambridge university


Topic #3:
lattice points non negative probability measure non zero square free loss generality number theory section prove linear combination fourier coeﬃcients


Topic #4:
quadratic forms binary quadratic weakly holomorphic holomorphic modular maass forms quadratic space section prove loss generality positive integer positive proportion


Topic #5:
eisenstein series modular forms siegel modular fourier expansion maass forms exact sequence holomorphic modular number theory fourier coeﬃcients square free


Topic #6:
mai

In [34]:
#Get similarities
N = M.todense()
distances = []
for ix in range(N.shape[0]):
    distances.append(cosine(N[-3], N[ix]))
print np.argsort(distances)[:10]
print np.around(sorted(distances)[:10],9)

[111 101  99  65  71 102  25  37  19  56]
[-0.          0.48054989  0.65083542  0.74890751  0.76693129  0.77536315
  0.7944557   0.79469272  0.83804994  0.8450582 ]


In [78]:
countvec = CountVectorizer(decode_error='ignore', stop_words=Stop, max_features=5000)
CV = countvec.fit_transform(new_text)

In [79]:

vocab=tuple(countvec.vocabulary_)

In [81]:
lda_model = lda.LDA(n_topics=10, n_iter=1500)
lda_model.fit(CV)

<lda.lda.LDA instance at 0x10ccb5878>

In [82]:
topic_word = lda_model.topic_word_  # model.components_ also works
n_top_words = 8
topic_words = []
for i, topic_dist in enumerate(topic_word):
    topic_words.append(np.array(vocab)[np.argsort(topic_dist)][:-n_top_words:-1])

In [83]:
for i in range(len(topic_words)):
    print ' '.join(topic_words[i])
    print '\n'

rest jγ approach plugging équation inductively extends


necessary hida simpler v2i projective simplex coordinate


monotonicity fractions helpful fil valid specok monogenic


probabilistic submatrix inspection 98 duality process weyl


williams border treatment implying unperturbed decimal submodule


au intersecting attacks p1p2 denominators generates tensoring


fractions qinghai area 4n inspection image 4ν2


jh 3b2 td thompson functional gives jk


approach inductively conductors landau milnor 54 coverings


diﬀeomorphism direct killing presented suﬃcient 65 10jδj




In [84]:
doc_topic = lda_model.doc_topic_

In [104]:
topic_most_pr = []
for n in range(10):
    topic_most_pr.append(doc_topic[n].argmax())

for n in range(10):
    print "doc: {} topic: {}".format(n, str(topic_words[topic_most_pr[n]]))

doc: 0 topic: [u'monotonicity' u'fractions' u'helpful' u'fil' u'valid' u'specok'
 u'monogenic']
doc: 1 topic: [u'monotonicity' u'fractions' u'helpful' u'fil' u'valid' u'specok'
 u'monogenic']
doc: 2 topic: [u'probabilistic' u'submatrix' u'inspection' u'98' u'duality' u'process'
 u'weyl']
doc: 3 topic: [u'rest' u'j\u03b3' u'approach' u'plugging' u'\xe9quation' u'inductively'
 u'extends']
doc: 4 topic: [u'rest' u'j\u03b3' u'approach' u'plugging' u'\xe9quation' u'inductively'
 u'extends']
doc: 5 topic: [u'williams' u'border' u'treatment' u'implying' u'unperturbed' u'decimal'
 u'submodule']
doc: 6 topic: [u'approach' u'inductively' u'conductors' u'landau' u'milnor' u'54'
 u'coverings']
doc: 7 topic: [u'monotonicity' u'fractions' u'helpful' u'fil' u'valid' u'specok'
 u'monogenic']
doc: 8 topic: [u'jh' u'3b2' u'td' u'thompson' u'functional' u'gives' u'jk']
doc: 9 topic: [u'monotonicity' u'fractions' u'helpful' u'fil' u'valid' u'specok'
 u'monogenic']


In [96]:
topic_most_pr

[2, 2, 3, 0, 0, 4, 8, 2, 7, 2]

In [102]:
topic_words[topic_most_pr[3]]

array([u'rest', u'j\u03b3', u'approach', u'plugging', u'\xe9quation',
       u'inductively', u'extends'], 
      dtype='<U17')